In [2]:
import pandas as pd

In [3]:
authors = pd.read_csv('../CSV Files/authors.csv', index_col='author_id')
authorship = pd.read_csv('../CSV Files/authorship.csv')
books = pd.read_csv('../CSV Files/books.csv', index_col='book_id')
book_tags = pd.read_csv('../CSV Files/book_tags.csv')
ratings = pd.read_csv('../CSV Files/ratings_with_date.csv')
tags = pd.read_csv('../CSV Files/tags.csv', index_col='tag_id')
to_read = pd.read_csv('../CSV Files/to_read_2.csv')
users = pd.read_csv('../CSV Files/user_2.csv')
users = users.drop(columns=['Unnamed: 0'], axis=1)

In [4]:
# convert str to dates
ratings['date'] = pd.to_datetime(ratings['date'])
to_read['date'] = pd.to_datetime(to_read['date'])
users['birthdate'] = pd.to_datetime(users['birthdate'])

# Create time date frame with time_id
time = pd.DataFrame()
time = pd.concat([ratings['date'], to_read['date'], users['birthdate']], ignore_index=True)
time = pd.DataFrame(time)
time.rename(columns={0: 'date'}, inplace=True)
time['flag'] = None
time.iloc[:5976479, [1]] = 'ratings_date'
time.iloc[5976479:6889184, [1]] = 'to_read_date'
time.iloc[6889184: , [1]] = 'users_date'
time['id'] = time.index + 1
time['year'] = time['date'].dt.year
time['month'] = time['date'].dt.month
time['day'] = time['date'].dt.day
#time = time[['date', 'year', 'month', 'day', 'flag']]
#time_id = time[['id', 'date', 'flag']]
#time_id['date'] = time_id['date'].astype(str)
time['date'] = time['date'].drop_duplicates()
time = time.dropna()
time = time.reset_index(drop=True)
time['id'] = time.index + 1



In [16]:
# create to_read with time_id
to_read_merged = pd.merge(to_read, time, on='date', how='inner')
to_read_merged = to_read_merged.drop(columns=['date', 'flag', 'year', 'month', 'day'])
to_read_merged.rename(columns={'id': 'time_id'}, inplace=True)

# Merge the two dataframes on the 'book_id' column
merged_df = pd.merge(to_read_merged, authorship, on='book_id', how='left')

# Extract the author_id for each book and add it as a new column in the ratings_merged dataframe
to_read_merged['author_id'] = merged_df['author_id']

to_read_merged

,user_id,book_id,time_id,author_id
0,9,8,2895,8
1,21459,8,2895,8
2,16629,177,2895,177
3,39496,297,2895,65
4,17941,1272,2895,1272
...,...,...,...,...
912700,46870,8685,5939,8685
912701,3576,5884,5940,167
912702,8472,5884,5941,167
912703,25722,5884,5942,167


In [13]:
# create ratings with time_id
ratings_merged = pd.merge(ratings, time, on='date', how='inner')
ratings_merged = ratings_merged.drop(columns=['date', 'flag', 'year', 'month', 'day'])
ratings_merged.rename(columns={'id': 'time_id'}, inplace=True)

# Merge the two dataframes on the 'book_id' column
merged_df = pd.merge(ratings_merged, authorship, on='book_id', how='left')

# Extract the author_id for each book and add it as a new column in the ratings_merged dataframe
ratings_merged['author_id'] = merged_df['author_id']

ratings_merged

,user_id,book_id,rating,time_id,author_id
0,1,258,5,1,258
1,89,2620,3,1,158
2,105,3832,3,1,1086
3,296,66,5,1,66
4,270,2076,3,1,464
...,...,...,...,...,...
5976474,51319,4200,3,3651,4200
5976475,15663,2475,2,3651,2475
5976476,52599,8685,1,3651,8685
5976477,30591,9137,3,3651,9137


In [14]:
# create users with time_id
users_merged = pd.merge(users, time, left_on='birthdate', right_on='date', how='inner')
users_merged = users_merged.drop(columns=['date', 'flag', 'year', 'month', 'day'])
users_merged.rename(columns={'id': 'time_id'}, inplace=True)

In [28]:
book_tags = pd.read_csv('../CSV Files/book_tags.csv')

books = pd.read_csv('../CSV Files/books.csv')
# Merge the two dataframes on the 'book_id' column
merged_df = pd.merge(book_tags, books, on='goodreads_book_id', how='left')

# Extract the author_id for each book and add it as a new column in the ratings_merged dataframe
book_tags['book_id'] = merged_df['book_id']

book_tags = book_tags.drop(columns = ['goodreads_book_id'], axis = 1)

authorship = pd.read_csv('../CSV Files/authorship.csv')
# Merge the two dataframes on the 'book_id' column
merged_df = pd.merge(book_tags, authorship, on='book_id', how='left')

# Extract the author_id for each book and add it as a new column in the ratings_merged dataframe
book_tags['author_id'] = merged_df['author_id']

book_tags.to_csv('../CSV Files/statistics_ratings.csv')

#Create a new dataframe with the following calculations: count of ratings, mean of ratings, and sum of ratings and sum of ratings 1-5
book_tags = book_tags.groupby(['book_id','tag_id', 'author_id']).agg({'count': 'sum'})

book_tags


count
book_id tag_id author_id       
1       698    1            358
        1120   1            339
        1128   1           1026
        1540   1           1263
        1542   1            492
...                         ...
10000   32900  10000          7
        32903  10000         13
        32905  10000         58
        32914  10000         96
        32915  10000          3

[999904 rows x 1 columns]

In [29]:
to_read_merged.to_csv('../CSV Files/to_read_merged.csv')
book_tags.to_csv('../CSV Files/book_tags.csv')
ratings_merged.to_csv('../CSV Files/ratings_merged.csv')
users_merged.to_csv('../CSV Files/users_merged.csv')
time.to_csv('../CSV Files/time.csv')